In [8]:
from typing import Dict, Text

import os
import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [121]:
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

ratings

<_MapDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [128]:
df = pd.read_csv(os.path.join('data','all_races_cleaned.csv'), index_col=0)

ratings = pd.DataFrame()
ratings['movie_title'] = df['race_class'].astype(str)
ratings['user_id'] = df['total_stakes'].astype(str)
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))
# ratings = tf.cast(ratings, tf.string)

movies = pd.DataFrame()
movies['movie_title'] = df['race_class'].astype(str)
movies = tf.data.Dataset.from_tensor_slices(dict(movies))
# movies = tf.cast(movies, tf.string)

movies

<_TensorSliceDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [129]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
# user_ids_vocabulary.adapt(ratings)

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies.map(lambda x: x["movie_title"]))

# ratings = tf.data.Dataset.from_tensor_slices(ratings)

In [143]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
#   def compute_loss(self, features, training=False) -> tf.Tensor:
    # Define how the loss is computed.
    # print(tf.cast(features[0], tf.int64))
    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])
    # user_embeddings = self.user_model(tf.cast(features, tf.int64))
    # movie_embeddings = self.movie_model(features)

    return self.task(user_embeddings, movie_embeddings)

In [144]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(lambda x: movie_model(x['movie_title']))
  )
)

In [146]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)


Epoch 1/3


6/6 [==============================] - 7s 967ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1965 - factorized_top_k/top_5_categorical_accuracy: 0.1965 - factorized_top_k/top_10_categorical_accuracy: 0.1965 - factorized_top_k/top_50_categorical_accuracy: 0.2102 - factorized_top_k/top_100_categorical_accuracy: 0.2167 - loss: 31712.0843 - regularization_loss: 0.0000e+00 - total_loss: 31712.0843
Epoch 2/3
6/6 [==============================] - 6s 907ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2271 - factorized_top_k/top_5_categorical_accuracy: 0.2271 - factorized_top_k/top_10_categorical_accuracy: 0.2271 - factorized_top_k/top_50_categorical_accuracy: 0.2397 - factorized_top_k/top_100_categorical_accuracy: 0.2402 - loss: 26259.2796 - regularization_loss: 0.0000e+00 - total_loss: 26259.2796
Epoch 3/3
6/6 [==============================] - 5s 880ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1934 - factorized_top_k/top_5_categorical_accuracy: 0.1934 - factori

In [148]:

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda x: (x['movie_title'], model.movie_model(x['movie_title']))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Top 3 recommendations for user 42: [b'griffin' b'griffin' b'griffin']
